# Análisis de los conntaminantes en la Ciudad de Méxcio a lo largo del día

In [66]:
import pandas as pd
import numpy as np

In [67]:
contaminantes_2016 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2016.csv")
contaminantes_2017 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2017.csv")
contaminantes_2018 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2018.csv")
contaminantes_2019 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2019.csv")
contaminantes_2020 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2020.csv")
contaminantes_2021 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2021.csv")
contaminantes_2022 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2022.csv")
contaminantes_2023 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2023.csv")
contaminantes_2024 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2024.csv")

df_unidades = pd.read_csv("../Datos/raw/catalogos/cat_unidades.csv", encoding="latin1")

In [68]:
import pandas as pd
import plotly.express as px
import datetime

# Unir todos los dataframes
dataframes = [contaminantes_2016, contaminantes_2017, contaminantes_2018, contaminantes_2019,
              contaminantes_2020, contaminantes_2021, contaminantes_2022, contaminantes_2023,
              contaminantes_2024]
df = pd.concat(dataframes)

# Reemplazo de '24:00' si es string
df['date'] = df['date'].apply(
    lambda x: x.replace('24:00', '00:00') if isinstance(x, str) else x
)

# Convertir a datetime y eliminar inválidos
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])

# Extraer año y hora
df['year'] = df['date'].dt.year
df['hour'] = df['date'].dt.strftime('%H:%M')

# Agrupar para promedio diario
df_avg = df.groupby(['id_parameter', 'year', 'hour'])['value'].mean().reset_index()

# Convertir 'hour' a datetime
df_avg['hour_dt'] = pd.to_datetime(df_avg['hour'], format='%H:%M')

# Obtener relación id_parameter -> unidad
param_to_unit = df[['id_parameter', 'unit']].drop_duplicates()
df_avg = df_avg.merge(param_to_unit, on='id_parameter', how='left')

# Agregar nombre de unidad
df_avg = df_avg.merge(df_unidades[['id_unidad', 'clave_unidad']], left_on='unit', right_on='id_unidad', how='left')

# Graficar
for param in df_avg['id_parameter'].unique():
    subset = df_avg[df_avg['id_parameter'] == param]
    unidad = subset['clave_unidad'].iloc[0] if not subset['clave_unidad'].isna().all() else ''

    fig = px.line(
        subset,
        x='hour_dt',
        y='value',
        color='year',
        title=f'Distribución Promedio Diaria - Contaminante {param}',
        labels={'value': f'Valor promedio ({unidad})', 'hour_dt': 'Hora del día', 'year': 'Año'},
        line_shape='spline',
    )

    fig.update_xaxes(
        dtick=3600000,  # 1 hora
        tickformat="%H:%M",
        title_text="Hora del día",
        tickangle=45
    )

    fig.update_layout(
        hovermode='x unified',
        template='plotly_white',
        legend_title_text='Año',
        title_x=0.5
    )

    fig.show()
